# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6637c32280>
├── 'a' --> tensor([[-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569]])
└── 'x' --> <FastTreeValue 0x7f6637c321c0>
    └── 'c' --> tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                        [-1.8371, -0.5802,  0.5726, -2.4750],
                        [ 0.3074,  1.6944, -0.2383, -0.0957]])

In [4]:
t.a

tensor([[-0.4974, -0.2386,  0.8347],
        [ 0.3685,  1.6074, -1.5569]])

In [5]:
%timeit t.a

64.1 ns ± 2.01 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6637c32280>
├── 'a' --> tensor([[ 0.1945, -1.5649, -1.6728],
│                   [ 0.7386,  0.4795,  0.5832]])
└── 'x' --> <FastTreeValue 0x7f6637c321c0>
    └── 'c' --> tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                        [-1.8371, -0.5802,  0.5726, -2.4750],
                        [ 0.3074,  1.6944, -0.2383, -0.0957]])

In [7]:
%timeit t.a = new_value

64.8 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569]]),
    x: Batch(
           c: tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                      [-1.8371, -0.5802,  0.5726, -2.4750],
                      [ 0.3074,  1.6944, -0.2383, -0.0957]]),
       ),
)

In [10]:
b.a

tensor([[-0.4974, -0.2386,  0.8347],
        [ 0.3685,  1.6074, -1.5569]])

In [11]:
%timeit b.a

61.4 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1046,  0.9024,  0.2541],
               [ 1.9528,  0.1195, -0.4990]]),
    x: Batch(
           c: tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                      [-1.8371, -0.5802,  0.5726, -2.4750],
                      [ 0.3074,  1.6944, -0.2383, -0.0957]]),
       ),
)

In [13]:
%timeit b.a = new_value

571 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

961 ns ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

265 µs ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

253 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f657de35ca0>
├── 'a' --> tensor([[[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]],
│           
│                   [[-0.4974, -0.2386,  0.8347],
│                    [ 0.3685,  1.6074, -1.5569]]])
└── 'x' --> <FastTreeValue 0x7f6584dcaa30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.1 µs ± 387 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f657de35f70>
├── 'a' --> tensor([[-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569],
│                   [-0.4974, -0.2386,  0.8347],
│                   [ 0.3685,  1.6074, -1.5569]])
└── 'x' --> <FastTreeValue 0x7f657de35f10>
    └── 'c' --> tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                        [-1.8371, -0.5802,  0.5726, -2.4750],
                 

In [23]:
%timeit t_cat(trees)

40.7 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

73.9 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]],
       
               [[-0.4974, -0.2386,  0.8347],
                [ 0.3685,  1.6074, -1.5569]]]),
    x: Batch(
           c: tensor([[[ 0.3289, -0.0880, -1.3438, -0.5130],
                       [-1.8371, -0.5802,  0.5726, -2.4750],
                       [ 0.3074,  1.6944, -0.2383, -0.0957]],
         

In [26]:
%timeit Batch.stack(batches)

93.4 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569],
               [-0.4974, -0.2386,  0.8347],
               [ 0.3685,  1.6074, -1.5569]]),
    x: Batch(
           c: tensor([[ 0.3289, -0.0880, -1.3438, -0.5130],
                      [-1.8371, -0.5802,  0.5726, -2.4750],
                      [ 0.3074,  1.6944, -0.2383, -0.0957],
                      [ 0.3289, -0.0880, -1.3438, -0.5130],
                      [-1.8371, -

In [28]:
%timeit Batch.cat(batches)

168 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

610 µs ± 213 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
